<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import numpy as np
import pandas as pd
import glob
import re

In [25]:
files = glob.glob('../transcripts_friends/season_all/*.csv')
files[:5]

['../transcripts_friends/season_all\\1001.csv',
 '../transcripts_friends/season_all\\1002.csv',
 '../transcripts_friends/season_all\\1003.csv',
 '../transcripts_friends/season_all\\1004.csv',
 '../transcripts_friends/season_all\\1005.csv']

In [24]:
df_total = pd.DataFrame(columns = ['Speaker','Text','Episode','Scene'])

for i in range(len(files)):
    df = pd.read_csv(files[i],error_bad_lines=False,sep=':', names = ['Speaker','Text'])
    
    df.Speaker = [i.lower() for i in df.Speaker]
    
    df = df.drop(df[(df.Speaker == 'written by')].index)
    df = df.drop(df[(df.Speaker == 'transcribed by')].index)
    df = df.drop(df[(df.Speaker == 'additional transcribing by')].index)
    df = df.drop(df[(df.Speaker == '(note')].index)
    df = df.drop(df[(df.Speaker == 'directed by')].index)
    df = df.drop(df[(df.Speaker == 'produced by')].index)
    df = df.drop(df[(df.Speaker == 'flashback clips transcribed by')].index)
    df = df.drop(df[(df.Speaker == 'teleplay by')].index)
    df = df.drop(df[(df.Speaker == 'story by')].index)
    
    df = df.drop(0)
    df['Episode'] = files[i][34:].split('.')[0]

    df = df.dropna().reset_index(drop=True)
    df.Text = [i.lower() for i in df.Text]

    scene_idx = list(df[df.Speaker == '[scene'].index)
    
    if scene_idx:
        df = df[scene_idx[0]:].reset_index(drop=True)

        scene_idx = list(df[df.Speaker == '[scene'].index)

        scenes = [a+b for a,b in zip(df.iloc[scene_idx].Speaker, df.iloc[scene_idx].Text)]
        scene_lengths = np.diff(scene_idx + [len(df)])
        df['Scene'] = sum([[scenes[i]] * scene_lengths[i] for i in range(len(scene_lengths))],[])

        df = df.drop(scene_idx)
    else:
        df['Scene'] = ['NaN'] * len(df)
        
    
    df_total = df_total.append(df,ignore_index = True)

In [28]:
df_total.to_csv('../data/All_Friends_data.csv')
df_total.to_pickle('../data/All_Friends_data.plk')

In [29]:
df_total

,Speaker,Text,Episode,Scene
0,monica,"oh, the way you crushed mike at ping pong was...",1001,"[scene barbados, monica and chandler's room. t..."
1,chandler,"you know, i'd love to, but i'm a little tired.",1001,"[scene barbados, monica and chandler's room. t..."
2,monica,i'll put a pillowcase over my head.,1001,"[scene barbados, monica and chandler's room. t..."
3,chandler,you're on!,1001,"[scene barbados, monica and chandler's room. t..."
4,phoebe,hey!,1001,"[scene barbados, monica and chandler's room. t..."
...,...,...,...,...
61315,ross,i'm sorry... we... we can't.,923-924,[scene hotel's bar. ross is running to charlie...
61316,charlie,"all right, all right.",923-924,[scene hotel's bar. ross is running to charlie...
61317,ross,"i mean, you just went out with my best friend...",923-924,[scene hotel's bar. ross is running to charlie...
61318,rachel,what?,923-924,[scene hotel's bar. ross is running to charlie...
